In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math
from flows.models import VARmodel

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# device_lib.list_local_devices()

In [3]:
!ls CDATA

AUS.csv  FRA.csv  GBR.csv  SYNTH.csv


In [4]:
ccodes = ['FRA']#, 'FRA', 'GBR']
datas = ['./CDATA/{}.csv'.format(x) for x in ccodes]

In [5]:
datas = [pd.read_csv(x, index_col='VARIABLE').values.astype(floatX).T[np.newaxis][:,1:-1] for x in datas]

In [6]:
[data.shape for data in datas]

[(1, 193, 4)]

In [7]:
scaler = 0.
for i, data in enumerate(datas):
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    scaler = scaler + stds
    datas[i] = data
print('---')
scaler /= len(datas)
for i in range(len(datas)):
    datas[i] /= scaler
    data = datas[i]
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
    datas[i] = data

[0.00571698 0.93158025 0.00503056 0.03016526]
---
[0.99999994 0.99999994 0.99999934 1.        ]


In [8]:
class VARmodel:
    def __init__(self, data, name='VARmodel', var_dim=None, mu=None, current_year=None):
        self.num_samples = 2000
        self.data_raw = data
        self.mu = mu
        self.var_dim = var_dim
        self.years = data.columns.values.astype('float32')
        years_c = tf.constant(data.columns.values, dtype=tf.float32, name='data_years')

        if current_year is None:
            self.OBSERV_STEPS = np.Infinity
        else:
            self.OBSERV_STEPS = tf.reduce_sum(tf.cast(years_c <= current_year, tf.int32))

        self.NUM_STEPS = data.shape[1]
        self.name = name
        self.logdensities = []
        self.priors = []
        self.dim = [self.var_dim,self.var_dim*2+1]
        self.summaries = []

        self.observable_mask = tf.range(0, self.NUM_STEPS, dtype=tf.int32) < self.OBSERV_STEPS

        pd = np.std(data.values[:,1:] - data.values[:,:-1], axis=-1).astype(floatX)[:self.var_dim]

        with tf.variable_scope(name, dtype=floatX) as scope:
            self.data = tf.get_variable(initializer=data.values.T[np.newaxis].astype(floatX),
                                    trainable=False, name='data')
            self.scope = scope

            self.create_rw_priors()
            self.outputs = self.create_walk_inference(mu=mu)
            self.create_observ_dispersion_inference(pd*0.5)
            self.create_likelihood(self.observable_mask, self.outputs)

    def create_summary(self, stype, name, tensor):
        s = stype(name, tensor)
        self.summaries.append(s)

    def create_rw_priors(self):
        dim = self.dim
        with tf.variable_scope('rw_priors'):
            s1 = 0.01/4
            cov_prior = Normal(dim=None, mu=0.5*math.log(s1), sigma=3.5, name='cov_prior')

            with tf.variable_scope('PWalk_inf'):
                with tf.variable_scope('flows'):
                    flow_conf = [NVPFlow(dim=self.dim[0]*self.dim[1], name='nvp_{}'.format(i)) for i in range(4)] + \
                        [LinearChol(dim=self.dim[0]*self.dim[1], name='lc')]
                    ldiag = DFlow(flow_conf, num_samples=self.num_samples)
                    ldiag.output += 0.5*math.log(s1)
                    ldiag.logdens -= tf.reduce_sum(ldiag.output, axis=-1)
                    print('ldiag logdens', ldiag.logdens)

                    self.logdensities.append(ldiag.logdens)
                
                if self.mu is None:
                    sigma0 = None
                else:
                    sigma0 = 3.
                    
                PWalk = MVNormalRW(dim=self.dim[0]*self.dim[1], 
                                   sigma0=3., 
                                   diag=tf.exp(ldiag.output), name='OrdWalk')
                self.priors.append(tf.reduce_sum(cov_prior.logdens(ldiag.output, reduce=False), axis=1))
                self.PWalk = PWalk
                tf.summary.scalar('s1_ord', tf.reduce_mean(tf.sqrt(PWalk.diag)))

    def create_walk_inference(self, mu=None):
        dim = self.dim
        gvar = GVAR(dim=dim[0]*dim[1], len=self.NUM_STEPS, name='coef_rw_inference', 
                    mu=mu, num_samples=self.num_samples)
        outputs = gvar.sample()

        self.logdensities.append(gvar.logdens)
        with tf.name_scope('PWald_prior'):
            pwld = self.PWalk.logdens(outputs)
            print('pwld', pwld)
            self.priors.append(tf.reduce_sum(pwld, axis=[1]))
        self.outputs = outputs

        return outputs

    def create_observ_dispersion_inference(self, prior_disp):
        print('Prior disp: {}'.format(prior_disp))
        prior_loc = -0.5*np.log(prior_disp).astype(floatX)

        with tf.variable_scope('obs_d_inf', reuse=tf.AUTO_REUSE):
            flow_conf = [NVPFlow(dim=self.var_dim, name='nvp_{}'.format(i)) for i in range(6)] + \
                [LinearChol(dim=self.var_dim, name='lc')]
            ldiag = DFlow(flow_conf, init_sigma=0.05, num_samples=self.num_samples)

            ldiag.output -= prior_loc
            ldiag.logdens -= tf.reduce_sum(ldiag.output, axis=-1)

        self.obs_d = tf.distributions.Normal(tf.constant(0., dtype=floatX), 
                                             scale=tf.exp(ldiag.output), name='obs_d_prior')
        
        with tf.name_scope('obsrv_prior'):
            pr = tf.reduce_sum(tf.distributions.Normal(loc=prior_loc[np.newaxis], 
                                                       scale=tf.constant(3., dtype=floatX)).log_prob(ldiag.output), 
                               axis=-1)
        tf.summary.scalar('mean_ods', tf.reduce_mean(tf.exp(ldiag.output)))
        self.logdensities.append(ldiag.logdens)
        self.priors.append(pr)

    def predict(self, observable_mask, outputs):
        dim = self.dim
        data = self.data
        out = tf.reshape(outputs, [self.num_samples, self.NUM_STEPS, dim[0], dim[1]])
        out = tf.transpose(out, [1,0,2,3])

        def step(prev, x):
            mask = x[0]
            prev_pred = tf.where(mask, x[1], prev)
            params = x[2]

            d0 = params[:,:,:dim[0]]
            d1 = params[:,:,dim[0]:2*dim[0]]

            pp1 = prev_pred[:,:dim[0]]
            pp0 = prev_pred[:,dim[0]:2*dim[0]]

            new_pred = tf.einsum('bij,bj->bi', d0, pp0) + tf.einsum('bij,bj->bi', d1, pp1)+ params[:,:,-1] + pp1
            obs_noise = self.obs_d.sample()
            new_pred = tf.where(mask, new_pred, new_pred + obs_noise)

            new_pred = tf.concat([new_pred, pp1], axis=1)
            return new_pred
        
        data = tf.transpose(tf.tile(data, [self.num_samples, 1, 1]), [1,0,2])
        ar = tf.scan(step, [observable_mask, data, out], 
                     initializer=tf.zeros([self.num_samples, 2*dim[0]], dtype=floatX))
        return ar

    def create_likelihood(self, observable_mask, outputs):
        dim = self.dim
        obs_d = self.obs_d

        preds = self.predict(observable_mask, outputs)
        self.preds = preds[:,:self.var_dim]
        
        with tf.name_scope('loglikelihood'):
            data = tf.transpose(self.data, [1,0,2])
            diffs = preds[:-1,:] - data[1:,:]
            diffs = diffs[:,:,:dim[0]]

            logl = tf.reduce_sum(obs_d.log_prob(diffs), [2])
            print('logl', logl)
            logl *= tf.cast(self.observable_mask[1:], floatX)[:,tf.newaxis]

            logl = tf.reduce_sum(logl, axis=0)
            self.priors.append(logl)


In [8]:
from flows.models import VARmodel

In [9]:
country_data = {c:d for c,d in zip(ccodes, datas)}

In [10]:
datas[0].shape

(1, 192, 8)

In [11]:
d = datas[0][0]
d = pd.DataFrame(d.T, columns=range(d.shape[0]))
datas[0] = d

In [12]:
with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
    model = VARmodel(datas[0], name='FRA', var_dim=4, current_year=3000.)

ldiag logdens Tensor("model/FRA/rw_priors/PWalk_inf/flows/sub_1:0", shape=(1024,), dtype=float32)
pwld Tensor("model/FRA/PWald_prior/OrdWalk/logdens/concat:0", shape=(1024, 192), dtype=float32)
Prior disp: [0.49601343 0.5011316  0.4979925  0.5013071 ]
preds Tensor("model/FRA/strided_slice:0", shape=(192, 1024, 4), dtype=float32)
blogl Tensor("model/FRA/loglikelihood/mul:0", shape=(191, 1024), dtype=float32)
logl Tensor("model/FRA/loglikelihood/Sum_1:0", shape=(1024,), dtype=float32)


In [13]:
model.priors

[<tf.Tensor 'model/FRA/rw_priors/PWalk_inf/Sum:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'model/FRA/PWald_prior/Sum:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'model/FRA/obsrv_prior/Sum:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'model/FRA/loglikelihood/Sum_1:0' shape=(1024,) dtype=float32>]

In [14]:
model.logdensities

[<tf.Tensor 'model/FRA/rw_priors/PWalk_inf/flows/sub_1:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'model/FRA/coef_rw_inference_1/VAR/logdens/add:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'model/FRA/obs_d_inf/sub_1:0' shape=(1024,) dtype=float32>]

In [15]:
prior = tf.add_n(model.priors)
logdensity = tf.add_n(model.logdensities)
print(prior, logdensity)

kl = logdensity - prior
kl = tf.reduce_mean(kl)
kl /= 36*160

lr = tf.constant(0.001)
main_op = tf.train.AdamOptimizer(lr).minimize(kl)

Tensor("AddN:0", shape=(1024,), dtype=float32) Tensor("AddN_1:0", shape=(1024,), dtype=float32)


In [16]:
kls = tf.summary.scalar('KLd', kl)
summary = tf.summary.merge_all()

In [30]:
model.PWalk.diag.eval().mean(axis=0)

array([0.0691975 , 0.23756538, 0.06122474, 0.18523894, 0.06376822,
       0.19053948, 0.07310431, 0.00921624, 0.31993693, 0.05851084,
       0.15029831, 0.05845614, 0.11992872, 0.0468922 , 0.14463224,
       0.1986526 , 0.01132096, 0.01153937, 0.08370733, 0.01362971,
       0.01045661, 0.01223395, 0.12885961, 0.01372957, 0.11040618,
       0.01482133, 0.01552367, 0.18504989, 0.01410815, 0.01324318,
       0.01765098, 0.02276933, 0.01394978, 0.02784467, 0.02197746,
       0.01521921], dtype=float32)

In [28]:
model.PWalk.diag.eval().std(axis=0)

array([0.00403532, 0.00914537, 0.00357073, 0.00753403, 0.00386657,
       0.00681846, 0.00473926, 0.00043602, 0.01328854, 0.00314246,
       0.00769496, 0.0032874 , 0.00421772, 0.00260856, 0.00556762,
       0.00874742, 0.00052711, 0.00052822, 0.00438387, 0.00057347,
       0.00063281, 0.00052898, 0.00624812, 0.00057629, 0.00559987,
       0.00063934, 0.0006772 , 0.00913313, 0.00061477, 0.00071913,
       0.00076966, 0.00106164, 0.00061931, 0.00134845, 0.00091614,
       0.0006671 ], dtype=float32)

In [18]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [19]:
init.run()

In [20]:
writer = tf.summary.FileWriter('/tmp/tfdbg/single')

In [21]:
epoch = 0

In [29]:
for epoch in range(epoch, 100000):
    print(epoch)
    for step in range(10):
        sess.run(main_op)
    s, _ = sess.run([summary, main_op], {lr: 0.001})
    writer.add_summary(s, global_step=epoch)

486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732


KeyboardInterrupt: 

In [33]:
preds = model.preds.eval().mean(axis=1)

In [40]:
dd = model.data_raw.values[:4,:].T

In [41]:
np.std(preds - dd, axis=0)

array([0.45195356, 0.66985625, 0.6350035 , 0.67699057], dtype=float32)

In [28]:
diags = []
for _ in range(4000):
    d = model.PWalk.diag.eval().mean(axis=0)
    diags.append(d)
diags = np.mean(diags, axis=0)

In [29]:
diags

array([0.00546182, 0.01633643, 0.01477936, 0.01048888, 0.01622361,
       0.01212972, 0.01796927, 0.00991377, 0.01518895, 0.01184873,
       0.0191168 , 0.01572438, 0.01451007, 0.01222456, 0.02210707,
       0.02022636, 0.0124552 , 0.01169597, 0.01707046, 0.01198687,
       0.01595444, 0.00960567, 0.03551232, 0.01234829, 0.03540852,
       0.01410678, 0.01352153, 0.01937663, 0.01219932, 0.02050923,
       0.01727278, 0.01346721, 0.01197882, 0.02661569, 0.02181612,
       0.01188068], dtype=float32)

In [ ]:
wsigmas = tf.sqrt(tf.diag_part(model.PWalk.sigma))

In [ ]:
wsigmas

In [ ]:
global_post=global_inf.output[0]

In [ ]:
ss = []
for _ in range(1000):
    ss.append(wsigmas.eval())
ss = np.array(ss)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
ss.std(axis=0)

In [ ]:
sns.kdeplot(ss[:,4], ss[:,1])
plt.show()